First, Python packages need to be imported for this project.

In [1]:
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline
import pandas as pd

Next, within the dataset chronicling all medals from all Olympic Games, those pertaining to 1960 through 2022 need to be selected.



In [2]:
# Initial data dump
col_list = ['discipline_title', 'slug_game', 'event_title', 
            'medal_type', 'country_name', 'country_3_letter_code']
medal_table = pd.read_csv('archive/olympic_medals.csv', usecols = col_list)

# Extract columns for hosting city and year from "slug_game" column
medal_table['City'] = medal_table['slug_game'].str[:-4].str.replace('-', ' ').str.strip().str.title()
medal_table['Year'] = medal_table['slug_game'].str[-4:]

# Format event titles and medal types
medal_table['event_title'] = medal_table['event_title'].str.title()
medal_table['medal_type'] = medal_table['medal_type'].str.lower().str.title()

# Clean medal table with organized data
new_col_list = ['Year', 'City', 'discipline_title', 'event_title', 
                'medal_type', 'country_name', 'country_3_letter_code']
new_medal_table = medal_table[new_col_list]

# Rename cleaned medal columns
new_medal_table = new_medal_table.rename(columns = {
    'discipline_title': 'Sport',
    'event_title': 'Event',
    'medal_type': 'Medal',
    'country_name': 'Country Name',
    'country_3_letter_code': 'Country Code'
})

In [3]:
new_medal_table

,Year,City,Sport,Event,Medal,Country Name,Country Code
0,2022,Beijing,Curling,Mixed Doubles,Gold,Italy,ITA
1,2022,Beijing,Curling,Mixed Doubles,Gold,Italy,ITA
2,2022,Beijing,Curling,Mixed Doubles,Silver,Norway,NOR
3,2022,Beijing,Curling,Mixed Doubles,Silver,Norway,NOR
4,2022,Beijing,Curling,Mixed Doubles,Bronze,Sweden,SWE
...,...,...,...,...,...,...,...
21692,1896,Athens,Weightlifting,Heavyweight - One Hand Lift Men,Silver,Denmark,DEN
21693,1896,Athens,Weightlifting,Heavyweight - One Hand Lift Men,Bronze,Greece,GRE
21694,1896,Athens,Weightlifting,Heavyweight - Two Hand Lift Men,Gold,Denmark,DEN
21695,1896,Athens,Weightlifting,Heavyweight - Two Hand Lift Men,Silver,Great Britain,GBR


The table is now compact and readable, but now needs only the rows that pertain to the Winter Olympics games from 1960 to 2022.

1964 &emsp; Innsbruck, Austria

1968 &emsp; Grenoble, France

1972 &emsp; Sapporo, Japan

1976 &emsp; Innsbruck, Austria

1980 &emsp; Lake Placid, New York, USA

1984 &emsp; Sarajevo, Bosnia and Herzegovina (Yugoslavia)

1988 &emsp; Calgary, Alberta, Canada

1992 &emsp; Albertville, France

1994 &emsp; Lillehamer, Norway

1998 &emsp; Nagano, Japan

2002 &emsp;Salt Lake City, Utah, USA

2006 &emsp; Turin, Italy

2010 &emsp; Vancouver, British Columbia, Canada

2014 &emsp; Sochi, Russia

2018 &emsp; Pyeongchang, South Korea

2022 &emsp; Beijing, China

In [4]:
# List the years and cities for the Winter Olympic games of interest
winter_years = ['1964', '1968', '1972', '1976','1980', '1984', '1988', '1992', 
                '1994', '1998', '2002', '2006', '2010', '2014', '2018', '2022']
                
winter_cities = ['Innsbruck', 'Grenoble', 'Sapporo', 'Innsbruck', 'Lake Placid',  'Sarajevo', 
                 'Calgary', 'Albertille', 'Lillehamer', 'Nagano',  'Salt Lake City', 'Turin', 
                 'Vancouver', 'Sochi', 'Pyeongchang', 'Beijing']

# Select the rows based on the above set of games
winter_medal_table = new_medal_table.loc[new_medal_table['Year'].isin(winter_years)]
winter_medal_table = winter_medal_table.loc[winter_medal_table['City'].isin(winter_cities)]

In [5]:
winter_medal_table

,Year,City,Sport,Event,Medal,Country Name,Country Code
0,2022,Beijing,Curling,Mixed Doubles,Gold,Italy,ITA
1,2022,Beijing,Curling,Mixed Doubles,Gold,Italy,ITA
2,2022,Beijing,Curling,Mixed Doubles,Silver,Norway,NOR
3,2022,Beijing,Curling,Mixed Doubles,Silver,Norway,NOR
4,2022,Beijing,Curling,Mixed Doubles,Bronze,Sweden,SWE
...,...,...,...,...,...,...,...
16012,1964,Innsbruck,Bobsleigh,Two-Man Men,Bronze,Italy,ITA
16013,1964,Innsbruck,Bobsleigh,Two-Man Men,Bronze,Italy,ITA
16014,1964,Innsbruck,Bobsleigh,Four-Man Men,Gold,Canada,CAN
16015,1964,Innsbruck,Bobsleigh,Four-Man Men,Silver,Austria,AUT


Now with the medal count data aggregated, it shall be important to clean the list of countries below in order to simplify it, since it consists of countries no longer in existence (such as the Soviet Union or Yugoslavia) and those that have for not very long (such as Belarus and Ukraine).

In [6]:
winter_medal_table['Country Name'].value_counts()

Norway                                   274
United States of America                 266
Germany                                  242
Canada                                   200
Austria                                  184
Soviet Union                             180
Switzerland                              135
Netherlands                              134
Sweden                                   129
German Democratic Republic (Germany)     129
Russian Federation                       114
France                                   114
Finland                                  112
Italy                                    112
People's Republic of China                78
Republic of Korea                         69
Japan                                     63
Federal Republic of Germany               48
ROC                                       38
Czech Republic                            33
Great Britain                             23
Slovenia                                  21
Poland    

That is why the three-letter country code is critical for comparing data in the Winter Olympic medal count (1960 - 2022) versus those which are currently evaluated by the World Bank for historic GDP.

In [7]:
# Initial data dump (note: data only extends to 2020 and will have to be extrapolated)
WB_country_list = ['Country Name', 'Country Code']
WB_year_list = ['1964', '1968', '1972', '1976','1980', '1984', '1988', '1992', 
                '1994', '1998', '2002', '2006', '2010', '2014', '2018', '2020']
WB_GDP = pd.read_csv(
    'API_NY.GDP.MKTP.CD_DS2_en_csv_v2_3930485/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_3930485.csv', 
    skiprows = 4,
    usecols = (WB_country_list + WB_year_list))

However, first the countries that are listed in the medal table need to have their codes verified in order to ensure that they are properly listed in the World Bank's economic data table.

In [8]:
# Create a list of the countries in the medal table to compare against the World Bank table
countries = winter_medal_table['Country Name'].value_counts().index[::].tolist()
codes = winter_medal_table['Country Code'].value_counts().index[::].tolist()
wrong = []

# Check which codes do not match
for index, code in enumerate(codes, 0):
    if WB_GDP['Country Code'].str.contains(code).any():
        pass
    else:
        wrong.append([countries[index], codes[index]])

In [9]:
wrong

[['Germany', 'GER'],
 ['Soviet Union', 'URS'],
 ['Switzerland', 'SUI'],
 ['Netherlands', 'NED'],
 ['German Democratic Republic (Germany)', 'GDR'],
 ['Federal Republic of Germany', 'FRG'],
 ['ROC', 'ROC'],
 ['Slovenia', 'SLO'],
 ['Czechoslovakia', 'TCH'],
 ['Olympic Athletes from Russia', 'OAR'],
 ['Latvia', 'LAT'],
 ['Croatia', 'CRO'],
 ['Bulgaria', 'BUL'],
 ['Yugoslavia', 'YUG'],
 ['Denmark', 'DEN']]

This issue needs to be nipped in the bud. Therefore, the countries listed above need their codes changed in order to have their GNI per capita data pulled. It is easier to apply these changes to the World Bank table, since each country has its own row.

Also, due to doping scandals among Russian athletes, the Russian Federation (RUS) has been banned from having an official delegation and hence had to compete as "Olympic Athletes from Russia" (OAR) in Pyeongchang 2018 and the "Russian Olympic Committee" in Beijing 2022. Despite this, it is just easier to merge these delegations into that of the Russian Federation.

In [10]:
# Change the country codes in the World Bank table to match that of the Winter Olympic table
for country in wrong:
    WB_GDP.loc[
        WB_GDP['Country Name'] == country[0], 'Country Code'] = country[1]

# Consolidate the entries in the Winter Olympic medal table for the Russian Federation
winter_medal_table['Country Name'].replace({'ROC': 'Russian Federation',
                                        'Olympic Athletes from Russia': 'Russian Federation'},
                                          inplace = True)
winter_medal_table['Country Code'].replace({'ROC': 'RUS', 'OAR': 'RUS'}, inplace = True)

# Change Liechtenstein to Austria to make things simpler
winter_medal_table['Country Name'].replace({'Liechtenstein': 'Austria'}, inplace = True)
winter_medal_table['Country Code'].replace({'LIE': 'AUT'}, inplace = True)

In [11]:
# Limit list only to countries that have scored medals in the Winter Olympics
WB_GDP = WB_GDP.loc[WB_GDP['Country Code'] \
                                          .isin(winter_medal_table['Country Code'])] \
                                          .sort_values(by='Country Code', ascending=True)

In [12]:
WB_GDP

,Country Name,Country Code,1964,1968,1972,1976,1980,1984,1988,1992,1994,1998,2002,2006,2010,2014,2018,2020
13,Australia,AUS,2.380110e+10,3.271699e+10,5.205140e+10,1.051015e+11,1.500323e+11,1.935936e+11,2.360659e+11,3.254803e+11,3.228073e+11,3.994045e+11,3.953427e+11,7.475562e+11,1.147589e+12,1.467505e+12,1.428530e+12,1.327836e+12
14,Austria,AUT,9.169984e+09,1.244063e+10,2.205961e+10,4.295998e+10,8.205891e+10,6.798534e+10,1.333394e+11,1.950781e+11,2.035352e+11,2.182599e+11,2.133778e+11,3.359986e+11,3.918927e+11,4.419961e+11,4.549459e+11,4.332585e+11
17,Belgium,BEL,1.596011e+10,2.137635e+10,3.720942e+10,7.111388e+10,1.268293e+11,8.334953e+10,1.622991e+11,2.347817e+11,2.448841e+11,2.585283e+11,2.571578e+11,4.079181e+11,4.809516e+11,5.346781e+11,5.430085e+11,5.218613e+11
25,Belarus,BLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.779300e+10,1.526437e+10,1.459425e+10,3.695431e+10,5.722249e+10,7.881384e+10,6.003126e+10,6.025824e+10
21,Bulgaria,BUL,NaN,NaN,NaN,NaN,1.983923e+10,1.759494e+10,2.255594e+10,1.035052e+10,9.697417e+09,1.503070e+10,1.640285e+10,3.437981e+10,5.068206e+10,5.708201e+10,6.636342e+10,6.988935e+10
35,Canada,CAN,4.937752e+10,7.182981e+10,1.130828e+11,2.065756e+11,2.738538e+11,3.553726e+11,5.073544e+11,5.923877e+11,5.781393e+11,6.340000e+11,7.606493e+11,1.319265e+12,1.617343e+12,1.805750e+12,1.725329e+12,1.645423e+12
40,China,CHN,5.970834e+10,7.084654e+10,1.136876e+11,1.539405e+11,1.911492e+11,2.599465e+11,3.123536e+11,4.269157e+11,5.643247e+11,1.029043e+12,1.470550e+12,2.752132e+12,6.087164e+12,1.047568e+13,1.389482e+13,1.472273e+13
99,Croatia,CRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.579288e+10,2.707455e+10,5.086079e+10,6.042602e+10,5.833029e+10,6.224787e+10,5.720378e+10
54,Czech Republic,CZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.480501e+10,4.785020e+10,6.680743e+10,8.219600e+10,1.562641e+11,2.090699e+11,2.093588e+11,2.489501e+11,2.453393e+11
58,Denmark,DEN,NaN,1.350557e+10,2.323238e+10,4.457589e+10,7.112753e+10,5.910524e+10,1.155528e+11,1.529157e+11,1.561624e+11,1.769919e+11,1.786352e+11,2.828849e+11,3.219953e+11,3.529936e+11,3.568412e+11,3.560849e+11


Obviously, due to the birth of new nations as well as the emergence of formerly communist ones (such as Poland and Romania) in the world scene result in a lack of GDP data for up until the early 1990s. Even data for Winter Olympic powerhouses such as Slovenia is missing up until the late 1900s, as well as some countries have missing data altogether (such North Korea and Liechtenstein at the present).

In order to conduct a more focused analysis of more recent Winter Olympic trends, it would be far simpler and more accurate to set the beginning of the study to 1998 in order to accommodate Slovenia as well as eliminate Liechtenstein (which is culturally similar to Austria and Switzerland) and North Korea (which is just a poor version of South Korea) entirely. Also, 1998 was the first year snowboarding was introduced into the Winter Olympics as an official sport with medals awarded.

In [13]:
# Trimmed down year list
year_list = ['1998', '2002', '2006', '2010', '2014', '2018', '2020']

# Limit list only to countries with reliable data as explained above
GDP = WB_GDP[WB_country_list + year_list].loc[
    (WB_GDP['Country Code'] != 'LIE') &
    (WB_GDP['Country Code'] != 'PRK')]

In [14]:
GDP

,Country Name,Country Code,1998,2002,2006,2010,2014,2018,2020
13,Australia,AUS,3.994045e+11,3.953427e+11,7.475562e+11,1.147589e+12,1.467505e+12,1.428530e+12,1.327836e+12
14,Austria,AUT,2.182599e+11,2.133778e+11,3.359986e+11,3.918927e+11,4.419961e+11,4.549459e+11,4.332585e+11
17,Belgium,BEL,2.585283e+11,2.571578e+11,4.079181e+11,4.809516e+11,5.346781e+11,5.430085e+11,5.218613e+11
25,Belarus,BLR,1.526437e+10,1.459425e+10,3.695431e+10,5.722249e+10,7.881384e+10,6.003126e+10,6.025824e+10
21,Bulgaria,BUL,1.503070e+10,1.640285e+10,3.437981e+10,5.068206e+10,5.708201e+10,6.636342e+10,6.988935e+10
35,Canada,CAN,6.340000e+11,7.606493e+11,1.319265e+12,1.617343e+12,1.805750e+12,1.725329e+12,1.645423e+12
40,China,CHN,1.029043e+12,1.470550e+12,2.752132e+12,6.087164e+12,1.047568e+13,1.389482e+13,1.472273e+13
99,Croatia,CRO,2.579288e+10,2.707455e+10,5.086079e+10,6.042602e+10,5.833029e+10,6.224787e+10,5.720378e+10
54,Czech Republic,CZE,6.680743e+10,8.219600e+10,1.562641e+11,2.090699e+11,2.093588e+11,2.489501e+11,2.453393e+11
58,Denmark,DEN,1.769919e+11,1.786352e+11,2.828849e+11,3.219953e+11,3.529936e+11,3.568412e+11,3.560849e+11


Now given this streamlined table, the final step shall be to extrapolate the GDP of the respective countries from 1998 to 2022 using the InterpolatedUnivariateSpline feature of the SciPy package. Use a linear curve to account for the fact that these mostly developed countries have a slower economic growth rate compared to that of developing countries.

In [15]:
# Initialize a new column as a list
new_col = []

# Loop through each country to extrapolate to the year 2022
for index, country in enumerate(GDP['Country Code'], 0):
    # Given values
    xi = np.array([int(x) for x in year_list])
    yi = np.array(GDP.loc[GDP['Country Code'] == country][year_list])
    # Positions to inter/extrapolate
    x = np.linspace(1998, 2022)
    # spline order: 1 linear, 2 quadratic, 3 cubic ... 
    order = 1
    # do inter/extrapolation
    s = InterpolatedUnivariateSpline(xi, yi, k = order)
    y = s(x)
    new_col.append(y[(len(y)) - 1])

# Extract columns for hosting city and year from "slug_game" column
GDP['2022'] = new_col

# Finally, drop the column for the year 2020
GDP = GDP.drop(columns = '2020', axis = 1)

In [16]:
GDP

,Country Name,Country Code,1998,2002,2006,2010,2014,2018,2022
13,Australia,AUS,3.994045e+11,3.953427e+11,7.475562e+11,1.147589e+12,1.467505e+12,1.428530e+12,1.227143e+12
14,Austria,AUT,2.182599e+11,2.133778e+11,3.359986e+11,3.918927e+11,4.419961e+11,4.549459e+11,4.115711e+11
17,Belgium,BEL,2.585283e+11,2.571578e+11,4.079181e+11,4.809516e+11,5.346781e+11,5.430085e+11,5.007141e+11
25,Belarus,BLR,1.526437e+10,1.459425e+10,3.695431e+10,5.722249e+10,7.881384e+10,6.003126e+10,6.048522e+10
21,Bulgaria,BUL,1.503070e+10,1.640285e+10,3.437981e+10,5.068206e+10,5.708201e+10,6.636342e+10,7.341527e+10
35,Canada,CAN,6.340000e+11,7.606493e+11,1.319265e+12,1.617343e+12,1.805750e+12,1.725329e+12,1.565518e+12
40,China,CHN,1.029043e+12,1.470550e+12,2.752132e+12,6.087164e+12,1.047568e+13,1.389482e+13,1.555064e+13
99,Croatia,CRO,2.579288e+10,2.707455e+10,5.086079e+10,6.042602e+10,5.833029e+10,6.224787e+10,5.215969e+10
54,Czech Republic,CZE,6.680743e+10,8.219600e+10,1.562641e+11,2.090699e+11,2.093588e+11,2.489501e+11,2.417285e+11
58,Denmark,DEN,1.769919e+11,1.786352e+11,2.828849e+11,3.219953e+11,3.529936e+11,3.568412e+11,3.553285e+11


Correspondingly, trim down the table of Winter Olympic Medals to that of the smaller range of years being studied (basically, the 21st century).

In [17]:
# Trimmed down year list now with 2022 properly at the end
new_year_list = ['1998', '2002', '2006', '2010', '2014', '2018', '2022']

# Select the rows based on the above set of games
winter_medal_table = winter_medal_table.loc[winter_medal_table['Year'].isin(new_year_list)]

In [18]:
winter_medal_table

,Year,City,Sport,Event,Medal,Country Name,Country Code
0,2022,Beijing,Curling,Mixed Doubles,Gold,Italy,ITA
1,2022,Beijing,Curling,Mixed Doubles,Gold,Italy,ITA
2,2022,Beijing,Curling,Mixed Doubles,Silver,Norway,NOR
3,2022,Beijing,Curling,Mixed Doubles,Silver,Norway,NOR
4,2022,Beijing,Curling,Mixed Doubles,Bronze,Sweden,SWE
...,...,...,...,...,...,...,...
8400,1998,Nagano,Short Track Speed Skating,500M Women,Silver,People's Republic of China,CHN
8401,1998,Nagano,Short Track Speed Skating,500M Women,Bronze,Republic of Korea,KOR
8402,1998,Nagano,Short Track Speed Skating,1000M Women,Gold,Republic of Korea,KOR
8403,1998,Nagano,Short Track Speed Skating,1000M Women,Silver,People's Republic of China,CHN


As a result, the medal count for each country has been greatly trimmed down to eliminate defunct and redundant countries.

In [19]:
winter_medal_table['Country Name'].value_counts()

Germany                       232
United States of America      204
Norway                        202
Canada                        177
Russian Federation            170
Austria                       128
Netherlands                    97
Sweden                         90
France                         88
Switzerland                    86
Italy                          80
People's Republic of China     78
Republic of Korea              69
Japan                          57
Finland                        56
Czech Republic                 33
Slovenia                       21
Poland                         19
Belarus                        18
Australia                      18
Great Britain                  17
Latvia                         14
Croatia                        11
Slovakia                       10
Estonia                         8
Ukraine                         8
Kazakhstan                      5
New Zealand                     5
Bulgaria                        5
Belgium       

The final level of cleaning needs to be performed in order to pick which names are simpler to present between the two tables

In [20]:
# Recreating the list of the countries from the trimmed Winter Olympics medal table
countries = winter_medal_table['Country Name'].value_counts().index[::].tolist()
codes = winter_medal_table['Country Code'].value_counts().index[::].tolist()
mismatch = []

# Check which codes do not match
for index, code in enumerate(codes, 0):
    if (GDP.loc[GDP['Country Code'] == code, 'Country Name']
        == countries[index]).any():
        pass
    else:
        mismatch.append([countries[index], 
                         GDP.loc[GDP['Country Code'] == code]
                                                ['Country Name']])

In [21]:
mismatch

[['United States of America',
  251    United States
  Name: Country Name, dtype: object],
 ["People's Republic of China",
  40    China
  Name: Country Name, dtype: object],
 ['Republic of Korea',
  126    Korea, Rep.
  Name: Country Name, dtype: object],
 ['Great Britain',
  81    United Kingdom
  Name: Country Name, dtype: object],
 ['Slovakia',
  221    Slovak Republic
  Name: Country Name, dtype: object]]

The simplest name is the best. Therefore, the following shall be implemented on both tables.

In [22]:
# Change country names in the Winter Olympic medal table
winter_medal_table['Country Name'].replace({'United States of America': 'United States'},
                                          inplace = True)
winter_medal_table['Country Name'].replace({"People's Republic of China": 'China'},
                                          inplace = True)
winter_medal_table['Country Name'].replace({'Republic of Korea': 'South Korea'},
                                          inplace = True)
winter_medal_table['Country Name'].replace({'Great Britain': 'United Kingdom'},
                                          inplace = True)


# Change country names in the World Bank table
GDP.loc[
    GDP['Country Name'] == 'Korea, Rep.', 'Country Name'] = 'South Korea'
GDP.loc[
    GDP['Country Name'] == 'Slovak Republic', 'Country Name'] = 'Slovakia'

/Users/jchatterjee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [23]:
winter_medal_table['Country Name'].value_counts()

Germany               232
United States         204
Norway                202
Canada                177
Russian Federation    170
Austria               128
Netherlands            97
Sweden                 90
France                 88
Switzerland            86
Italy                  80
China                  78
South Korea            69
Japan                  57
Finland                56
Czech Republic         33
Slovenia               21
Poland                 19
Belarus                18
Australia              18
United Kingdom         17
Latvia                 14
Croatia                11
Slovakia               10
Estonia                 8
Ukraine                 8
Kazakhstan              5
New Zealand             5
Bulgaria                5
Belgium                 4
Hungary                 4
Spain                   3
Denmark                 1
Name: Country Name, dtype: int64

Finally, trim down the list of countries that have won medals during the years 2002 to 2022.

In [24]:
# Limit list only to countries that have scored medals in the Winter Olympics
GDP = GDP.loc[GDP['Country Code'] \
              .isin(winter_medal_table['Country Code'])] \
              .sort_values(by='Country Code', ascending = True)

# Set index to country name
GDP.index = GDP['Country Name']
GDP = GDP.drop(columns = ['Country Name', 'Country Code'])

In [25]:
GDP

,1998,2002,2006,2010,2014,2018,2022
Country Name,,,,,,,
Australia,3.994045e+11,3.953427e+11,7.475562e+11,1.147589e+12,1.467505e+12,1.428530e+12,1.227143e+12
Austria,2.182599e+11,2.133778e+11,3.359986e+11,3.918927e+11,4.419961e+11,4.549459e+11,4.115711e+11
Belgium,2.585283e+11,2.571578e+11,4.079181e+11,4.809516e+11,5.346781e+11,5.430085e+11,5.007141e+11
Belarus,1.526437e+10,1.459425e+10,3.695431e+10,5.722249e+10,7.881384e+10,6.003126e+10,6.048522e+10
Bulgaria,1.503070e+10,1.640285e+10,3.437981e+10,5.068206e+10,5.708201e+10,6.636342e+10,7.341527e+10
Canada,6.340000e+11,7.606493e+11,1.319265e+12,1.617343e+12,1.805750e+12,1.725329e+12,1.565518e+12
China,1.029043e+12,1.470550e+12,2.752132e+12,6.087164e+12,1.047568e+13,1.389482e+13,1.555064e+13
Croatia,2.579288e+10,2.707455e+10,5.086079e+10,6.042602e+10,5.833029e+10,6.224787e+10,5.215969e+10
Czech Republic,6.680743e+10,8.219600e+10,1.562641e+11,2.090699e+11,2.093588e+11,2.489501e+11,2.417285e+11


Since the time frame is rather short and the countries listed are relatively political stable (ignoring the present situation in Eastern Europe), there is no need to factor in inflation for the GDP figures.

In [26]:
# Initial data dump (note: data only extends to 2020 and will have to be extrapolated)
pop_country_list = ['Country Name', 'Country Code']
pop_year_list = ['1998', '2002', '2006', '2010', '2014', '2018', '2020']
population = pd.read_csv(
    'API_SP.POP.TOTL_DS2_en_csv_v2_3918567/API_SP.POP.TOTL_DS2_en_csv_v2_3918567.csv', 
    skiprows = 4,
    usecols = (pop_country_list + pop_year_list))

# Change country names in the World Bank table
population.loc[
    population['Country Name'] == 'Korea, Rep.', 'Country Name'] = 'South Korea'
population.loc[
    population['Country Name'] == 'Slovak Republic', 'Country Name'] = 'Slovakia'

# Change the country codes in the World Bank table to match that of the Winter Olympic table
for country in wrong:
    population.loc[
        population['Country Name'] == country[0], 'Country Code'] = country[1]

# Limit list only to countries that have scored medals in the Winter Olympics
population = population.loc[population['Country Code'] \
                                          .isin(winter_medal_table['Country Code'])] \
                                          .sort_values(by='Country Code', ascending = True)

# Initialize a new column as a list
new_col = []

# Loop through each country to extrapolate to the year 2022
for index, country in enumerate(population['Country Code'], 0):
    # Given values
    xi = np.array([int(x) for x in year_list])
    yi = np.array(population.loc[population['Country Code'] == country][year_list])
    # Positions to inter/extrapolate
    x = np.linspace(1998, 2022)
    # spline order: 1 linear, 2 quadratic, 3 cubic ... 
    order = 1
    # do inter/extrapolation
    s = InterpolatedUnivariateSpline(xi, yi, k = order)
    y = s(x)
    new_col.append(y[(len(y)) - 1])

# Extract columns for hosting city and year from "slug_game" column
population['2022'] = new_col

# Finally, drop the column for the year 2020
population = population.drop(columns = '2020', axis = 1)

# Set index to country name
population.index = population['Country Name']
population = population.drop(columns = ['Country Name', 'Country Code'])

In [27]:
population.style.format('{:,.0f}')

,1998,2002,2006,2010,2014,2018,2022
Country Name,,,,,,,
Australia,"18,711,000","19,651,400","20,697,900","22,031,750","23,475,686","24,982,688","26,391,394"
Austria,"7,976,789","8,081,957","8,268,641","8,363,404","8,546,356","8,840,521","8,993,889"
Belgium,"10,203,008","10,332,785","10,547,958","10,895,586","11,209,057","11,427,054","11,684,940"
Belarus,"10,071,963","9,865,548","9,604,924","9,483,836","9,448,515","9,438,785","9,321,119"
Bulgaria,"8,256,786","7,837,161","7,601,022","7,395,599","7,223,938","7,025,037","6,842,993"
Canada,"30,155,173","31,360,079","32,571,174","34,004,889","35,437,435","37,065,178","38,945,298"
China,"1,241,935,000","1,280,400,000","1,311,020,000","1,337,705,000","1,371,860,000","1,402,760,000","1,419,098,724"
Croatia,"4,532,135","4,302,174","4,311,159","4,295,427","4,238,389","4,087,843","4,006,557"
Czech Republic,"10,294,373","10,196,916","10,238,905","10,474,410","10,525,347","10,629,928","10,767,864"


After the GDP and population have been tabulated, the next step is to calculate the GDP per capita.

In [28]:
GDP_per_capita = (GDP / population).round(decimals = 2)

In [29]:
GDP_per_capita.style.format("{:,.0f}")

,1998,2002,2006,2010,2014,2018,2022
Country Name,,,,,,,
Australia,"21,346","20,118","36,117","52,088","62,512","57,181","46,498"
Austria,"27,362","26,402","40,635","46,858","51,718","51,461","45,761"
Belgium,"25,338","24,888","38,673","44,142","47,701","47,520","42,851"
Belarus,"1,516","1,479","3,847","6,034","8,341","6,360","6,489"
Bulgaria,"1,820","2,093","4,523","6,853","7,902","9,447","10,729"
Canada,"21,025","24,255","40,504","47,562","50,956","46,549","40,198"
China,829,"1,149","2,099","4,550","7,636","9,905","10,958"
Croatia,"5,691","6,293","11,797","14,068","13,762","15,228","13,019"
Czech Republic,"6,490","8,061","15,262","19,960","19,891","23,420","22,449"


First, general medal count data shall be tallied. The actual weighting of the medal does not matter, but only the overall quantity of them regardless of whether they were gold, silver, or bronze.

In [30]:
medal_count = population.copy()

In [31]:
for col in medal_count.columns:
    medal_count[col].values[:] = 0

In [33]:
olympiads = new_year_list

for index, year in enumerate(olympiads, 0):
    # Find medals in each year
    medals = winter_medal_table[winter_medal_table['Year'].isin([year])]
    # Count values
    medals = medals['Country Name'].value_counts()
    # Create dataframe
    medals = medals.to_frame()
    # Extract values and index
    medals_values = medals['Country Name']
    medals_index = medals.index
    # Dump values into the table
    medal_count.loc[medals_index, year] = medals_values
    # Convert to int
    medal_count[year] = medal_count[year].astype('int')

In [34]:
medal_count

,1998,2002,2006,2010,2014,2018,2022
Country Name,,,,,,,
Australia,1,2,2,3,3,3,4
Austria,17,17,24,17,18,16,19
Belgium,1,0,0,0,0,1,2
Belarus,2,1,1,3,6,3,2
Bulgaria,1,3,1,0,0,0,0
Canada,16,18,26,29,27,34,27
China,8,9,13,13,9,10,16
Croatia,0,4,3,3,1,0,0
Czech Republic,3,3,4,6,8,7,2


Next, the medals shall be differentiated by event.

In [ ]:
winter_medal_table['Sport'].value_counts()

Clean the data to merge "Short Track" with "Short Track Speed Skating".

In [ ]:
# Change sport names in the Winter Olympic medal table
winter_medal_table['Sport'].replace({'Short Track': 'Short Track Speed Skating'},
                                          inplace = True)
winter_medal_table['Sport'].replace({'Speed skating': 'Long Track Speed Skating'},
                                          inplace = True)

In [ ]:
winter_medal_table['Sport'].value_counts()

In [ ]:
# Select the rows based on the above set of games
winter_medal_table.loc[winter_medal_table['Sport'].isin(['Cross Country Skiing'])
                      & winter_medal_table['Year'].isin(['2002'])]['Country Name'].value_counts()